# SMPL-X: Expressive Body Capture
Download the model files from [here](https://smpl-x.is.tue.mpg.de) and place them in the `parametric_models` folder.
After logging in, go to the "Download" tab and "Download SMPL-X v1.1 (NPZ+PKL, 830 MB)".

The packages required for this is same as before in addition to these packages:
```bash
tqdm
imageio[ffmpeg]
```

In [2]:
from smplx import SMPLX
from tqdm import tqdm
import numpy as np
import torch
import trimesh
import pyrender
import imageio
import os

In [3]:
SMPLX_MODEL_PATH = "./parametric_models/smplx"

smplx_model = SMPLX(
    model_path=SMPLX_MODEL_PATH,
    gender='neutral',
    use_pca=False,
    num_betas=10,
    num_expression_coeffs=100,
    flat_hand_mean=True
)

### Using the model with a real mocap data.

In [4]:
data = np.load('sample_smplx.npz')
list(data.keys())

['poses',
 'trans',
 'betas',
 'expressions',
 'model',
 'gender',
 'mocap_frame_rate']

In [5]:
betas = data['betas']
poses = data['poses']
expressions = data['expressions']
transl = data['trans']

print("Betas shape:", betas.shape)
print("Poses shape:", poses.shape)
print("Expressions shape:", expressions.shape)
print("Translation shape:", transl.shape)

Betas shape: (10,)
Poses shape: (1016, 165)
Expressions shape: (1016, 100)
Translation shape: (1016, 3)


In [6]:
names_dict = {
    0: 'pelvis',
    1: 'left_hip',
    2: 'right_hip',
    3: 'spine1',
    4: 'left_knee',
    5: 'right_knee',
    6: 'spine2',
    7: 'left_ankle',
    8: 'right_ankle',
    9: 'spine3',
    10: 'left_foot',
    11: 'right_foot',
    12: 'neck',
    13: 'left_collar',
    14: 'right_collar',
    15: 'head',
    16: 'left_shoulder',
    17: 'right_shoulder',
    18: 'left_elbow',
    19: 'right_elbow',
    20: 'left_wrist',
    21: 'right_wrist',
    22: 'jaw',
    23: 'left_eye_smplhf',
    24: 'right_eye_smplhf',
    25: 'left_index1',
    26: 'left_index2',
    27: 'left_index3',
    28: 'left_middle1',
    29: 'left_middle2',
    30: 'left_middle3',
    31: 'left_pinky1',
    32: 'left_pinky2',
    33: 'left_pinky3',
    34: 'left_ring1',
    35: 'left_ring2',
    36: 'left_ring3',
    37: 'left_thumb1',
    38: 'left_thumb2',
    39: 'left_thumb3',
    40: 'right_index1',
    41: 'right_index2',
    42: 'right_index3',
    43: 'right_middle1',
    44: 'right_middle2',
    45: 'right_middle3',
    46: 'right_pinky1',
    47: 'right_pinky2',
    48: 'right_pinky3',
    49: 'right_ring1',
    50: 'right_ring2',
    51: 'right_ring3',
    52: 'right_thumb1',
    53: 'right_thumb2',
    54: 'right_thumb3',
    55: 'nose',
    56: 'right_eye',
    57: 'left_eye',
    58: 'right_ear',
    59: 'left_ear',
    60: 'left_big_toe',
    61: 'left_small_toe',
    62: 'left_heel',
    63: 'right_big_toe',
    64: 'right_small_toe',
    65: 'right_heel',
    66: 'left_thumb',
    67: 'left_index',
    68: 'left_middle',
    69: 'left_ring',
    70: 'left_pinky',
    71: 'right_thumb',
    72: 'right_index',
    73: 'right_middle',
    74: 'right_ring',
    75: 'right_pinky',
    76: 'right_eye_brow1',
    77: 'right_eye_brow2',
    78: 'right_eye_brow3',
    79: 'right_eye_brow4',
    80: 'right_eye_brow5',
    81: 'left_eye_brow5',
    82: 'left_eye_brow4',
    83: 'left_eye_brow3',
    84: 'left_eye_brow2',
    85: 'left_eye_brow1',
    86: 'nose1',
    87: 'nose2',
    88: 'nose3',
    89: 'nose4',
    90: 'right_nose_2',
    91: 'right_nose_1',
    92: 'nose_middle',
    93: 'left_nose_1',
    94: 'left_nose_2',
    95: 'right_eye1',
    96: 'right_eye2',
    97: 'right_eye3',
    98: 'right_eye4',
    99: 'right_eye5',
    100: 'right_eye6',
    101: 'left_eye4',
    102: 'left_eye3',
    103: 'left_eye2',
    104: 'left_eye1',
    105: 'left_eye6',
    106: 'left_eye5',
    107: 'right_mouth_1',
    108: 'right_mouth_2',
    109: 'right_mouth_3',
    110: 'mouth_top',
    111: 'left_mouth_3',
    112: 'left_mouth_2',
    113: 'left_mouth_1',
    114: 'left_mouth_5',
    115: 'left_mouth_4',
    116: 'mouth_bottom',
    117: 'right_mouth_4',
    118: 'right_mouth_5',
    119: 'right_lip_1',
    120: 'right_lip_2',
    121: 'lip_top',
    122: 'left_lip_2',
    123: 'left_lip_1',
    124: 'left_lip_3',
    125: 'lip_bottom',
    126: 'right_lip_3'
}

In [33]:
from scipy.spatial.transform import Rotation as R

def z_up_to_y_up(v):
    v_matrix = R.from_rotvec(v)
    z_to_y_rotation = R.from_euler('x', -90, degrees=True)
    return (z_to_y_rotation * v_matrix).as_rotvec()

global_orient = torch.from_numpy(z_up_to_y_up(poses[:, :3])).float()

body_pose = torch.from_numpy(poses[:, 3:66]).float()
jaw_pose = torch.from_numpy(poses[:, 66:69]).float()
leye_pose = torch.from_numpy(poses[:, 69:72]).float()
reye_pose = torch.from_numpy(poses[:, 72:75]).float()
left_hand_pose = torch.from_numpy(poses[:, 75:120]).float()
right_hand_pose = torch.from_numpy(poses[:, 120:165]).float()


expressions_tensor = torch.from_numpy(expressions).float()
betas_tensor = torch.from_numpy(betas[:10]).reshape(1, 10).float()
transl_tensor = torch.from_numpy(transl).float()


smplx_out = smplx_model(
                    global_orient=global_orient,
                    body_pose=body_pose,
                    betas=betas_tensor,
                    left_hand_pose=left_hand_pose,
                    right_hand_pose=right_hand_pose,
                    jaw_pose=jaw_pose,
                    leye_pose=leye_pose,
                    reye_pose=reye_pose,
                    expression=expressions_tensor,
                    transl=transl_tensor
                )
vertices = smplx_out.vertices
joints = smplx_out.joints
print("Vertices shape:", vertices.shape)
print("Joints shape:", joints.shape)

Vertices shape: torch.Size([1016, 10475, 3])
Joints shape: torch.Size([1016, 127, 3])


In [34]:
os.environ["PYOPENGL_PLATFORM"] = "egl"

def render(vertices, faces, image_size=(800, 800)):
    vertices = vertices[0].cpu().numpy()

    # ---- Build mesh ----
    mesh = trimesh.Trimesh(vertices=vertices,
                           faces=faces,
                           process=False)
    pm = pyrender.Mesh.from_trimesh(mesh, smooth=True)

    # ---- Scene ----
    scene = pyrender.Scene(bg_color=np.zeros(4))
    # Add mesh
    scene.add(pm)
    
    # ---- Auto center ----
    center = mesh.bounds.mean(axis=0)
    extents = mesh.extents
    radius = np.linalg.norm(extents) * 0.5

    # ---- Camera ----
    camera = pyrender.PerspectiveCamera(yfov=np.deg2rad(45.0))
    scene.add(
        camera,
        pose=np.array([
            [1, 0, 0, center[0]],
            [0, 1, 0, center[1]],
            [0, 0, 1, center[2] + 2.5 * radius],
            [0, 0, 0, 1],
        ])
    )

    # ---- Light (simple directional) ----
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
    scene.add(
        light,
        pose=np.array([
            [1, 0, 0, center[0]],
            [0, 1, 0, center[1] + radius],
            [0, 0, 1, center[2] + radius],
            [0, 0, 0, 1],
        ])
    )

    # ---- Render ----
    w, h = image_size
    r = pyrender.OffscreenRenderer(w, h)
    color, _ = r.render(scene)
    r.delete()

    return color

In [36]:
frames = []
for i in tqdm(range(300, 600, 3)):
    img = render(vertices[i:i+1], smplx_model.faces)
    frames.append(img)

imageio.mimwrite('smplx_mocap.mp4', frames, fps=10)

100%|██████████| 100/100 [00:17<00:00,  5.57it/s]
